# Meeting July 23 nd

## GPU Kernels for Matrix-free

Basically just the translation of the CPU code

Implementation for D2x or differental operators in x direction is fairly easy with nice performance. Speed-up compared to CPU iterative code is roughly 100x. Memory throughput is over 100 GB/s. Jeremy's well-optimzed GPU copy kernel can achieve something around 220 GB/s with shared_memory and tiling, so I guess this performance is close to optimal.

But the problem is in the y direction. In GPU code, speed is mostly affected by how your optimize memory assignment. In operators associated with y directions, different threads will be working on non-adjacent grid point data, and it's very in-efficient. (the speedup decreases when the size of the matrix increases) I can't think of how to use tiling with shared memory in GPU copy and rotate kernel here.

One approach is to restack u in a different direction and use D2x for D2y, then restack it back. Theoretically it's just a matrix rotation, and you can use shared memory for that, but doing so for each u twice is still far from efficient 






In [5]:
# include("deriv_ops.jl")

using DataFrames
using CUDA
using Printf
using StaticArrays
using GPUifyLoops: @unroll


function D2x(u, Nx, Ny, h)
	N = Nx*Ny
	y = zeros(N)
	idx = 1:Ny
	y[idx] = (u[idx] - 2 .* u[Ny .+ idx] + u[2*Ny .+ idx]) ./ h^2

	idx1 = Ny+1:N-Ny
	y[idx1] = (u[idx1 .- Ny] - 2 .* u[idx1] + u[idx1 .+ Ny]) ./ h^2

	idx2 = N-Ny+1:N
	y[idx2] = (u[idx2 .- 2*Ny] -2 .* u[idx2 .- Ny] + u[idx2]) ./ h^2

	return y
end

function D2x_GPU(d_u, d_y, Nx, Ny, h, ::Val{TILE_DIM}) where {TILE_DIM}
	tidx = (blockIdx().x - 1) * TILE_DIM + threadIdx().x
	N = Nx*Ny
	# d_y = zeros(N)
	if tidx <= Ny
		d_y[tidx] = (d_u[tidx] - 2 * d_u[Ny + tidx] + d_u[2*Ny + tidx]) / h^2
	end
	sync_threads()

	if Ny+1 <= tidx <= N-Ny
		d_y[tidx] = (d_u[tidx - Ny] - 2 .* d_u[tidx] + d_u[tidx + Ny]) / h^2
	end

	sync_threads()

	if N-Ny+1 <= tidx <= N
		d_y[tidx] = (d_u[tidx - 2*Ny] -2 * d_u[tidx - Ny] + d_u[tidx]) / h^2
	end

	nothing
end

function D2x_GPU_v2(d_u, d_y, Nx, Ny, h, ::Val{TILE_DIM}) where {TILE_DIM}
	tidx = (blockIdx().x - 1) * TILE_DIM + threadIdx().x
	N = Nx*Ny
	# d_y = zeros(N)
	if tidx <= Ny
		d_y[tidx] = (d_u[tidx] - 2 * d_u[Ny + tidx] + d_u[2*Ny + tidx]) / h^2
	end

	if Ny+1 <= tidx <= N-Ny
		d_y[tidx] = (d_u[tidx - Ny] - 2 .* d_u[tidx] + d_u[tidx + Ny]) / h^2
	end


	if N-Ny+1 <= tidx <= N
		d_y[tidx] = (d_u[tidx - 2*Ny] -2 * d_u[tidx - Ny] + d_u[tidx]) / h^2
	end

	sync_threads()

	nothing
end


function tester_D2x(Nx)
	# Nx = Ny = 1000;
	Ny = Nx
	u = randn(Nx * Ny)
	d_u = CuArray(u)
	d_y = similar(d_u)
	h = 1/Nx
	TILE_DIM=32
	t1 = 0
	t2 = 0
	t3 = 0

	rep_times = 10

	THREAD_NUM = 32
	BLOCK_NUM = div(Nx * Ny,TILE_DIM) + 1

	y = D2x(u,Nx,Ny,h)
	@cuda threads=THREAD_NUM blocks=BLOCK_NUM D2x_GPU(d_u,d_y,Nx,Ny,h,Val(TILE_DIM))
	y_gpu = collect(d_y)
	@cuda threads=THREAD_NUM blocks=BLOCK_NUM D2x_GPU_v2(d_u,d_y,Nx,Ny,h,Val(TILE_DIM))
	y_gpu_2 = collect(d_y)
	@show y ≈ y_gpu
	@show y ≈ y_gpu_2

	ty = time_ns()
	for i in 1:rep_times
		y = D2x(u,Nx,Ny,h)
	end
	ty_end = time_ns()
	t1 = ty_end - ty
	t_dy = time_ns()
	for i in 1:rep_times
		@cuda threads=THREAD_NUM blocks=BLOCK_NUM D2x_GPU(d_u,d_y,Nx,Ny,h,Val(TILE_DIM))
	end
	synchronize()
	# sync_threads()
	t_dy_end = time_ns()
	t2 = t_dy_end - t_dy

	t_dy_v2 = time_ns()
	for i in 1:rep_times
		@cuda threads=THREAD_NUM blocks=BLOCK_NUM D2x_GPU_v2(d_u,d_y,Nx,Ny,h,Val(TILE_DIM))
	end
	synchronize()
	# sync_threads()
	t_dy_v2_end = time_ns()
	t3 = t_dy_v2_end - t_dy_v2

	@show Float64(t1)
	@show Float64(t2)
	@show Float64(t3)

	@show t1/t2
	@show t1/t3

	memsize = length(u) * sizeof(eltype(u))
	@printf("CPU Through-put %20.2f\n", 2 * memsize * rep_times / t1)
	@printf("GPU Through-put %20.2f\n", 2 * memsize * rep_times / t2)
	@printf("GPU (v2) Through-put %20.2f\n", 2 * memsize * rep_times / t3)

	return Float64(t1), Float64(t2), Float64(t3)
end

tester_D2x (generic function with 1 method)

In [7]:
tester_D2x(100)

y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 1.739299e6
Float64(t2) = 312900.0
Float64(t3) = 271800.0
t1 / t2 = 5.558641738574624
t1 / t3 = 6.399186902133922
CPU Through-put                 0.92
GPU Through-put                 5.11
GPU (v2) Through-put                 5.89


(1.739299e6, 312900.0, 271800.0)

In [8]:
tester_D2x(500)

y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 5.0142301e7
Float64(t2) = 833000.0
Float64(t3) = 660600.0
t1 / t2 = 60.19483913565426
t1 / t3 = 75.90417953375719
CPU Through-put                 0.80
GPU Through-put                48.02
GPU (v2) Through-put                60.55


(5.0142301e7, 833000.0, 660600.0)

In [9]:
tester_D2x(1000)

y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 3.55542001e8
Float64(t2) = 2.1365e6
Float64(t3) = 2.0478e6
t1 / t2 = 166.41329323660193
t1 / t3 = 173.62144789530228
CPU Through-put                 0.45
GPU Through-put                74.89
GPU (v2) Through-put                78.13


(3.55542001e8, 2.1365e6, 2.0478e6)

In [10]:
tester_D2x(10000)

y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 2.11824347e10
Float64(t2) = 1.71425101e8
Float64(t3) = 1.338631e8
t1 / t2 = 123.5667039216153
t1 / t3 = 158.23953501749176
CPU Through-put                 0.76
GPU Through-put                93.34
GPU (v2) Through-put               119.53


(2.11824347e10, 1.71425101e8, 1.338631e8)

Also results from previous run-time

```
julia> tester_D2x(1000)
y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 1.95963799e8
Float64(t2) = 2.2188e6
Float64(t3) = 2.006499e6
t1 / t2 = 88.31972192175951
t1 / t3 = 97.66453858187819
CPU Through-put                 0.82
GPU Through-put                72.11
GPU (v2) Through-put                79.74
(1.95963799e8, 2.2188e6, 2.006499e6)

julia> tester_D2x(2000)
y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 1.0486394e9
Float64(t2) = 8.002999e6
Float64(t3) = 7.609599e6
t1 / t2 = 131.03080482704047
t1 / t3 = 137.8048173103471
CPU Through-put                 0.61
GPU Through-put                79.97
GPU (v2) Through-put                84.10
(1.0486394e9, 8.002999e6, 7.609599e6)

julia> tester_D2x(10000)
y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 2.40318696e10
Float64(t2) = 1.807678e8
Float64(t3) = 1.349505e8
t1 / t2 = 132.94330959385465
t1 / t3 = 178.07914457523313
CPU Through-put                 0.67     
GPU Through-put                88.51     
GPU (v2) Through-put               118.56
(2.40318696e10, 1.807678e8, 1.349505e8)
```

In [13]:

function D2y(u, Nx, Ny, h)
	N = Nx*Ny
	y = zeros(N)
	idx = 1:Ny:N-Ny+1
	y[idx] = (u[idx] - 2 .* u[idx .+ 1] + u[idx .+ 2]) ./ h^2

	idx1 = Ny:Ny:N
	y[idx1] = (u[idx1 .- 2] - 2 .* u[idx1 .- 1] + u[idx1]) ./ h^2

	for j = 1:Nx
		idx = 2+(j-1)*Ny:j*Ny-1
		y[idx] = (u[idx .- 1] - 2 .* u[idx] + u[idx .+ 1]) ./ h^2
	end

	return y

end

function D2y_GPU(d_u, d_y, Nx, Ny, h, ::Val{TILE_DIM}) where {TILE_DIM}
	# tidx = ((blockIdx().x - 1) * TILE_DIM + threadIdx().x - 1)*Ny + 1
	tidx = (blockIdx().x - 1) * TILE_DIM + threadIdx().x
	N = Nx*Ny
	# d_y = zeros(N)
	t1 = (tidx - 1)*Ny + 1
	if 1 <= t1 <= N - Ny + 1
		d_y[t1] = (d_u[t1] - 2 * d_u[t1 + 1] + d_u[t1 + 2]) / h^2
	end
	sync_threads()

	t2 = tidx * Ny
	if Ny <= t2 <= N
		d_y[t2] = (d_u[t2 - 2] - 2 * d_u[t2 - 1] + d_u[t2]) / h^2
	end

	sync_threads()

	for j = 1:Nx
		if 2 + (j-1)*Ny <= tidx <= j*Ny-1
			d_y[tidx] = (d_u[tidx - 1] - 2 * d_u[tidx] + d_u[tidx + 1]) / h^2
		end
	end
	sync_threads()

	# if N-Ny+1 <= tidx <= N
	# 	d_y[tidx] = (d_u[tidx - 2*Ny] -2 * d_u[tidx - Ny] + d_u[tidx]) / h^2
	# end

	nothing
end

function D2y_GPU_v2(d_u, d_y, Nx, Ny, h, ::Val{TILE_DIM}) where {TILE_DIM}
	# tidx = ((blockIdx().x - 1) * TILE_DIM + threadIdx().x - 1)*Ny + 1
	tidx = (blockIdx().x - 1) * TILE_DIM + threadIdx().x
	N = Nx*Ny
	# d_y = zeros(N)
	t1 = (tidx - 1)*Ny + 1
	if 1 <= t1 <= N - Ny + 1
		d_y[t1] = (d_u[t1] - 2 * d_u[t1 + 1] + d_u[t1 + 2]) / h^2
	end


	t2 = tidx * Ny
	if Ny <= t2 <= N
		d_y[t2] = (d_u[t2 - 2] - 2 * d_u[t2 - 1] + d_u[t2]) / h^2
	end

	for j = 1:Nx
		if 2 + (j-1)*Ny <= tidx <= j*Ny-1
			d_y[tidx] = (d_u[tidx - 1] - 2 * d_u[tidx] + d_u[tidx + 1]) / h^2
		end
	end

	# if N-Ny+1 <= tidx <= N
	# 	d_y[tidx] = (d_u[tidx - 2*Ny] -2 * d_u[tidx - Ny] + d_u[tidx]) / h^2
	# end

	nothing
end

function tester_D2y(Nx)
	# Nx = Ny = 1000;
	Ny = Nx
	u = randn(Nx * Ny)
	d_u = CuArray(u)
	d_y = similar(d_u)
	h = 1/Nx
	TILE_DIM=32
	t1 = 0
	t2 = 0
	t3 = 0

	rep_times = 10

	THREAD_NUM = 32
	BLOCK_NUM = div(Nx * Ny,TILE_DIM) + 1

	y = D2y(u,Nx,Ny,h)
	@cuda threads=THREAD_NUM blocks=BLOCK_NUM D2y_GPU(d_u,d_y,Nx,Ny,h,Val(TILE_DIM))
	y_gpu = collect(d_y)
	@cuda threads=THREAD_NUM blocks=BLOCK_NUM D2y_GPU_v2(d_u,d_y,Nx,Ny,h,Val(TILE_DIM))
	y_gpu_2 = collect(d_y)
	@show y ≈ y_gpu
	@show y ≈ y_gpu_2

	ty = time_ns()
	for i in 1:rep_times
		y = D2x(u,Nx,Ny,h)
	end
	ty_end = time_ns()
	t1 = ty_end - ty
	t_dy = time_ns()
	for i in 1:rep_times
		@cuda threads=THREAD_NUM blocks=BLOCK_NUM D2y_GPU(d_u,d_y,Nx,Ny,h,Val(TILE_DIM))
	end
	synchronize()
	# sync_threads()
	t_dy_end = time_ns()
	t2 = t_dy_end - t_dy

	t_dy_v2 = time_ns()
	for i in 1:rep_times
		@cuda threads=THREAD_NUM blocks=BLOCK_NUM D2y_GPU_v2(d_u,d_y,Nx,Ny,h,Val(TILE_DIM))
	end
	synchronize()
	# sync_threads()
	t_dy_v2_end = time_ns()
	t3 = t_dy_v2_end - t_dy_v2

	@show Float64(t1)
	@show Float64(t2)
	@show Float64(t3)

	@show t1/t2
	@show t1/t3

	memsize = length(u) * sizeof(eltype(u))
	@printf("CPU Through-put %20.2f\n", 2 * memsize * rep_times / t1)
	@printf("GPU Through-put %20.2f\n", 2 * memsize * rep_times / t2)
	@printf("GPU (v2) Through-put %20.2f\n", 2 * memsize * rep_times / t3)

	return Float64(t1), Float64(t2), Float64(t3)
end


tester_D2y (generic function with 1 method)

In [14]:
tester_D2y(100)

y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 2.3381e6
Float64(t2) = 425900.0
Float64(t3) = 339799.0
t1 / t2 = 5.489786334820381
t1 / t3 = 6.8808324921497706
CPU Through-put                 0.68
GPU Through-put                 3.76
GPU (v2) Through-put                 4.71


(2.3381e6, 425900.0, 339799.0)

In [15]:
tester_D2y(500)

y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 4.17032e7
Float64(t2) = 1.16499e7
Float64(t3) = 1.12927e7
t1 / t2 = 3.5797045468201443
t1 / t3 = 3.6929343735333444
CPU Through-put                 0.96
GPU Through-put                 3.43
GPU (v2) Through-put                 3.54


(4.17032e7, 1.16499e7, 1.12927e7)

In [17]:
tester_D2y(1000)

y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 1.72581301e8
Float64(t2) = 7.91476e7
Float64(t3) = 6.68384e7
t1 / t2 = 2.180499484507427
t1 / t3 = 2.5820681075549383
CPU Through-put                 0.93
GPU Through-put                 2.02
GPU (v2) Through-put                 2.39


(1.72581301e8, 7.91476e7, 6.68384e7)

In [18]:
tester_D2y(10000)

y ≈ y_gpu = true
y ≈ y_gpu_2 = true
Float64(t1) = 2.07101086e10
Float64(t2) = 6.1000083299e10
Float64(t3) = 6.06608891e10
t1 / t2 = 0.3395095134294597
t1 / t3 = 0.34140793033645134
CPU Through-put                 0.77
GPU Through-put                 0.26
GPU (v2) Through-put                 0.26


(2.07101086e10, 6.1000083299e10, 6.06608891e10)

## Results from C++ Implementations from these two blogs

### In X direction 
https://developer.nvidia.com/blog/finite-difference-methods-cuda-cc-part-1/<br>

### In Y Z direction
https://developer.nvidia.com/blog/finite-difference-methods-cuda-cc-part-1/<br>
![GPU Results](graphics/GPU_finite_difference.jpg)